In [13]:
from astropy.coordinates import Angle
import pandas as pd
import numpy as np

from km3irf import irf_tools 

In [4]:
df = pd.read_csv("temp_dataset.csv")

In [7]:
len(df)

200

In [8]:
df.head()

,E,dir_x,dir_y,dir_z,E_mc,dir_x_mc,dir_y_mc,dir_z_mc,weight_w2,bdt0,bdt1
0,349.621209,-0.670299,0.270036,0.691216,268.990,-0.670996,0.286741,0.683772,4.510000e+11,12.0,-2.000000
1,20.423436,0.327411,0.078130,-0.941646,178.710,0.327387,0.079789,-0.941515,1.182000e+11,1.0,-2.000000
2,184.966630,0.794867,0.354793,-0.492248,214.610,0.799260,0.353143,-0.486285,1.960000e+11,1.0,-2.000000
3,52.327015,0.455668,-0.632999,0.625843,291.347,0.432161,-0.625802,0.649314,5.622000e+11,11.0,0.998906
4,248.662668,0.553978,-0.036197,0.831744,299.805,0.835802,-0.326768,0.441200,5.829000e+11,10.0,-2.000000


Now it needs to fill all bins for PSF

In [17]:
# create bins
cos_theta_binE = np.linspace(1, -1, 7)
energy_binE = np.logspace(2, 8, 25)
rad_binE = np.concatenate(
    (
        np.linspace(0, 1, 20, endpoint=False),
        np.linspace(1, 5, 40, endpoint=False),
        np.linspace(5, 30, 51),
        [180.0],
    )
)

theta_binE = np.arccos(cos_theta_binE)
# Bin centers
energy_binC = np.sqrt(energy_binE[:-1] * energy_binE[1:])
theta_binC = np.arccos(0.5 * (cos_theta_binE[:-1] + cos_theta_binE[1:]))
rad_binC = 0.5 * (rad_binE[1:] + rad_binE[:-1])

weights_array = np.ones(len(df), dtype=np.float64)

In [56]:
len(rad_binC)

111

In [40]:
psf = irf_tools.psf_3D(e_bins=energy_binE, t_bins=theta_binE, r_bins=rad_binE, dataset=df, weights=weights_array)
psf.sum()

AttributeError: 'numpy.ndarray' object has no attribute 'axis'

In [ ]:
# rad_max = Angle(np.linspace(0, self.rad[-1].value, 10 * len(self.rad)), "rad")




In [21]:
import astropy.units as u

In [23]:
rad_deg = rad_binE * u.deg

In [55]:
rad = rad_deg.to('rad')
len(rad)

112

In [57]:
rad_C_deg = rad_binC * u.deg
rad_C = rad_C_deg.to('rad')

In [39]:
# increase number of rad bins in 10 times
rad_max = Angle(np.linspace(0, rad[-1].value, 10 * len(rad)), unit="rad")
rad[-1].value

3.141592653589793

In [35]:
np.linspace(1, 10, 101)

array([ 1.  ,  1.09,  1.18,  1.27,  1.36,  1.45,  1.54,  1.63,  1.72,
        1.81,  1.9 ,  1.99,  2.08,  2.17,  2.26,  2.35,  2.44,  2.53,
        2.62,  2.71,  2.8 ,  2.89,  2.98,  3.07,  3.16,  3.25,  3.34,
        3.43,  3.52,  3.61,  3.7 ,  3.79,  3.88,  3.97,  4.06,  4.15,
        4.24,  4.33,  4.42,  4.51,  4.6 ,  4.69,  4.78,  4.87,  4.96,
        5.05,  5.14,  5.23,  5.32,  5.41,  5.5 ,  5.59,  5.68,  5.77,
        5.86,  5.95,  6.04,  6.13,  6.22,  6.31,  6.4 ,  6.49,  6.58,
        6.67,  6.76,  6.85,  6.94,  7.03,  7.12,  7.21,  7.3 ,  7.39,
        7.48,  7.57,  7.66,  7.75,  7.84,  7.93,  8.02,  8.11,  8.2 ,
        8.29,  8.38,  8.47,  8.56,  8.65,  8.74,  8.83,  8.92,  9.01,
        9.1 ,  9.19,  9.28,  9.37,  9.46,  9.55,  9.64,  9.73,  9.82,
        9.91, 10.  ])

In [41]:
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])

# Sum along axis 0 (columns)
sum_axis_0 = np.sum(arr, axis=0)

# Sum along axis 1 (rows)
sum_axis_1 = np.sum(arr, axis=1)

print("Sum along axis 0:", sum_axis_0)
print("Sum along axis 1:", sum_axis_1)

Sum along axis 0: [12 15 18]
Sum along axis 1: [ 6 15 24]


In [42]:
psf_value = np.sum(psf, axis=(1,2))

In [49]:
rad1 = np.atleast_1d(rad_max)

astropy.coordinates.angles.Angle

In [50]:
rad1 == rad_max

array([ True,  True,  True, ...,  True,  True,  True])

In [52]:
import scipy
from km3irf.interpolation import ScaledRegularGridInterpolator

In [53]:
def interpolate(rad, psf_value):
        points = (rad,)
        return ScaledRegularGridInterpolator(
            points=points, values=psf_value
        )

In [63]:
1 * u.GeV

<Quantity 1. GeV>

In [ ]:
def evaluate(energy, rad, method="linear"):
        """Evaluate the PSF at a given energy and offset
        """
        energy = np.atleast_1d(u.Quantity(energy))[:, np.newaxis]
        rad = np.atleast_1d(u.Quantity(rad))
        return self.interpolate((energy, rad), clip=True, method=method)


In [ ]:
def interpolate_containment(rad, energy):
        # if rad[0] > 0:
        #     rad = self.rad.insert(0, 0)
        # else:
        #     rad = self.rad

        rad_drad = 2 * np.pi * rad * self.evaluate(energy=self.energy, rad=rad)
        values = scipy.integrate.cumtrapz(
            rad_drad.to_value("rad-1"), rad.to_value("rad"), initial=0, axis=1
        )

        points = (self.energy, rad)
        return ScaledRegularGridInterpolator(points=points, values=values, fill_value=1)

In [68]:
# Generate example data
x = np.linspace(0, 1, 3)
y = np.linspace(0, 1, 4)
z = np.linspace(0, 1, 5)

# Create a grid of points
xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')

# Generate values for the grid points
values = np.sin(xx) * np.cos(yy) * np.exp(-zz)

# Create an instance of ScaledRegularGridInterpolator
interp = ScaledRegularGridInterpolator(
    points=(x, y, z),
    values=values,
    points_scale=['lin', 'lin', 'lin'],  # Linear scaling for x, y, z
    values_scale='lin'  # Linear scaling for values
)

# Generate new points for interpolation
x_new = np.linspace(0, 1, 10)
y_new = np.linspace(0, 1, 11)
z_new = np.linspace(0, 1, 12)

# Create a grid of new points
xx_new, yy_new, zz_new = np.meshgrid(x_new, y_new, z_new, indexing='ij')

# Interpolate the values at the new points
interpolated_values = interp((xx_new, yy_new, zz_new))

# Print the interpolated values for demonstration
print("Interpolated Values:")
print(interpolated_values)

Interpolated Values:
[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.10653901 0.09796943 0.08939985 ... 0.04728948 0.04324149 0.03919351]
  [0.10477974 0.09635167 0.0879236  ... 0.04650859 0.04252745 0.03854631]
  [0.10302047 0.09473391 0.08644734 ... 0.0457277  0.04181341 0.03789911]
  ...
  [0.0732619  0.067369   0.0614761  ... 0.03251876 0.02973516 0.02695155]
  [0.06541259 0.06015105 0.05488952 ... 0.02903469 0.02654932 0.02406395]
  [0.05756327 0.05293311 0.04830294 ... 0.02555061 0.02336348 0.02117634]]

 [[0.21307802 0.19593886 0.1787997  ... 0.09457895 0.08648299 0.07838702]
 

In [69]:
xx


array([[[0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. ]],

       [[0.5, 0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5, 0.5]],

       [[1. , 1. , 1. , 1. , 1. ],
        [1. , 1. , 1. , 1. , 1. ],
        [1. , 1. , 1. , 1. , 1. ],
        [1. , 1. , 1. , 1. , 1. ]]])